In [1]:
import requests
from bs4 import BeautifulSoup

# URL of the Cricsheet matches page
url = 'https://cricsheet.org/matches/'

# Send a GET request to fetch the page content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all anchor tags with href attributes containing '.json'
json_links = [a['href'] for a in soup.find_all('a', href=True) if '.json' in a['href']]

# Download each JSON file
for link in json_links:
    # Construct the full URL
    full_url = f'https://cricsheet.org{link}'
    # Send a GET request to fetch the JSON file
    json_response = requests.get(full_url)
    # Extract the filename from the URL
    filename = link.split('/')[-1]
    # Save the JSON file
    with open(f'data/raw/{filename}', 'wb') as f:
        f.write(json_response.content)
    print(f'Downloaded: {filename}')


In [10]:
import os
import json
import pandas as pd
import logging
import traceback
from sqlalchemy import create_engine   # optional for DB save

# set a logger to capture parsing issues
logging.basicConfig(filename='cricsheet_parse.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s: %(message)s')

def normalize_string(s):
    if s is None:
        return None
    return str(s).strip()

def parse_single_match(file_path, match_type):
    """
    Parse one Cricsheet JSON file into:
      - one match-level dict
      - list of delivery (ball-level) dicts
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    info = data.get('info', {}) or {}

    match_id = os.path.splitext(os.path.basename(file_path))[0]

    # date handling
    dates = info.get('dates')
    if isinstance(dates, list) and len(dates) > 0:
        date = dates[0]
    elif isinstance(dates, str):
        date = dates
    else:
        date = None

    city = info.get('city')
    venue = info.get('venue')
    teams = info.get('teams') or []
    toss = info.get('toss') or {}
    outcome = info.get('outcome') or {}

    # outcome can be complex: normalize to runs/wickets fields
    winner = outcome.get('winner')
    result_type = outcome.get('result')  # e.g., 'tie', 'draw' sometimes
    by = outcome.get('by')
    result_by_runs = None
    result_by_wickets = None
    if isinstance(by, dict):
        result_by_runs = by.get('runs')
        result_by_wickets = by.get('wickets')
    else:
        # keep raw string if sometimes it's a string
        pass

    match_row = {
        'match_id': match_id,
        'match_type': match_type,
        'date': date,
        'city': city,
        'venue': venue,
        'team1': teams[0] if len(teams) > 0 else None,
        'team2': teams[1] if len(teams) > 1 else None,
        'toss_winner': toss.get('winner'),
        'toss_decision': toss.get('decision'),
        'winner': winner,
        'result_type': result_type,
        'result_by_runs': result_by_runs,
        'result_by_wickets': result_by_wickets
    }

    deliveries = []
    innings_list = data.get('innings', []) or []

    for inning in innings_list:
        # handle two possible structures:
        # 1) inning is a dict with keys 'team' and 'overs'
        # 2) inning is like {'1st innings': { 'team':..., 'overs':[...] }}
        if 'overs' not in inning:
            # try to find the nested dict
            inner = None
            for v in inning.values():
                if isinstance(v, dict):
                    inner = v
                    break
            if inner:
                inning_dict = inner
            else:
                # unexpected structure -> skip
                logging.warning(f"Unexpected inning structure in {file_path}: {inning}")
                continue
        else:
            inning_dict = inning

        team = inning_dict.get('team')
        for over in inning_dict.get('overs', []) or []:
            over_number = over.get('over')
            deliveries_list = over.get('deliveries') or []

            for delivery in deliveries_list:
                # delivery may be either:
                # - dict with fields directly: {'batter':..., 'bowler':..., 'runs': {...}}
                # - or a wrapper like {'0.1': { ...fields... }}
                if 'batter' in delivery or 'bowler' in delivery or 'runs' in delivery:
                    d = delivery
                    ball_id = None
                else:
                    # attempt to extract nested dict (e.g., {'0.1': {...}})
                    try:
                        # pick first value that is a dict
                        first_val = next(iter(delivery.values()))
                        if isinstance(first_val, dict):
                            ball_id = next(iter(delivery.keys()))
                            d = first_val
                        else:
                            d = delivery
                            ball_id = None
                    except Exception:
                        d = delivery
                        ball_id = None

                runs = d.get('runs') or {}
                runs_batter = runs.get('batter', 0)
                runs_extras = runs.get('extras', 0)
                runs_total = runs.get('total', runs_batter + runs_extras)

                wicket_obj = d.get('wicket') if isinstance(d.get('wicket'), dict) else None
                wicket_kind = wicket_obj.get('kind') if wicket_obj else None
                player_out = wicket_obj.get('player_out') if wicket_obj else None

                deliveries.append({
                    'match_id': match_id,
                    'match_type': match_type,
                    'date': date,
                    'city': city,
                    'team': team,
                    'over': over_number,
                    'ball_id': ball_id,   # may be None or '0.1' etc
                    'batter': normalize_string(d.get('batter')),
                    'non_striker': normalize_string(d.get('non_striker')),
                    'bowler': normalize_string(d.get('bowler')),
                    'runs_batter': runs_batter,
                    'runs_extras': runs_extras,
                    'runs_total': runs_total,
                    'wicket_kind': wicket_kind,
                    'player_out': normalize_string(player_out)
                })

    return match_row, deliveries

def parse_folder(folder_path, match_type):
    matches = []
    deliveries = []
    files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

    for fname in files:
        fp = os.path.join(folder_path, fname)
        try:
            mrow, drows = parse_single_match(fp, match_type)
            matches.append(mrow)
            deliveries.extend(drows)
        except Exception as e:
            logging.error(f"Failed parsing {fp}: {e}\n{traceback.format_exc()}")
            continue

    matches_df = pd.DataFrame(matches)
    deliveries_df = pd.DataFrame(deliveries)
    return matches_df, deliveries_df


In [11]:
# update these paths to your local folders
folders = {
    'IPL': r"C:\py project\cricsheet_project\data\Raw\IPL",
    'ODI': r"C:\py project\cricsheet_project\data\Raw\ODI",
    'T20': r"C:\py project\cricsheet_project\data\Raw\T20",
    'TEST': r"C:\py project\cricsheet_project\data\Raw\TEST"
}

all_matches = []
all_deliveries = []

for mtype, folder in folders.items():
    if not os.path.isdir(folder):
        print(f"Folder not found: {folder} (skipping {mtype})")
        continue
    print(f"Parsing folder: {folder} as {mtype}")
    m_df, d_df = parse_folder(folder, mtype)
    print(f"  -> matches: {m_df.shape[0]} | deliveries: {d_df.shape[0]}")
    all_matches.append(m_df)
    all_deliveries.append(d_df)

matches_df = pd.concat(all_matches, ignore_index=True) if all_matches else pd.DataFrame()
deliveries_df = pd.concat(all_deliveries, ignore_index=True) if all_deliveries else pd.DataFrame()


Parsing folder: C:\py project\cricsheet_project\data\Raw\IPL as IPL
  -> matches: 1169 | deliveries: 278205
Parsing folder: C:\py project\cricsheet_project\data\Raw\ODI as ODI
  -> matches: 3000 | deliveries: 1588920
Parsing folder: C:\py project\cricsheet_project\data\Raw\T20 as T20
  -> matches: 4570 | deliveries: 1034413
Parsing folder: C:\py project\cricsheet_project\data\Raw\TEST as TEST
  -> matches: 879 | deliveries: 1700455


In [12]:
print("Matches table shape:", matches_df.shape)
display(matches_df.head())

print("\nDeliveries table shape:", deliveries_df.shape)
display(deliveries_df.head())

# missing values summary
print("\nMatches missing values:")
print(matches_df.isnull().sum())

print("\nDeliveries missing values (top):")
print(deliveries_df.isnull().sum().sort_values(ascending=False).head(20))


Matches table shape: (9618, 13)


,match_id,match_type,date,city,venue,team1,team2,toss_winner,toss_decision,winner,result_type,result_by_runs,result_by_wickets
0,1082591,IPL,2017-04-05,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,Sunrisers Hyderabad,None,35.0,NaN
1,1082592,IPL,2017-04-06,Pune,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians,Rising Pune Supergiant,field,Rising Pune Supergiant,None,NaN,7.0
2,1082593,IPL,2017-04-07,Rajkot,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,Kolkata Knight Riders,None,NaN,10.0
3,1082594,IPL,2017-04-08,Indore,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant,Kings XI Punjab,field,Kings XI Punjab,None,NaN,6.0
4,1082595,IPL,2017-04-08,Bengaluru,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,None,15.0,NaN



Deliveries table shape: (4601993, 15)


,match_id,match_type,date,city,team,over,ball_id,batter,non_striker,bowler,runs_batter,runs_extras,runs_total,wicket_kind,player_out
0,1082591,IPL,2017-04-05,Hyderabad,Sunrisers Hyderabad,0,None,DA Warner,S Dhawan,TS Mills,0,0,0,None,None
1,1082591,IPL,2017-04-05,Hyderabad,Sunrisers Hyderabad,0,None,DA Warner,S Dhawan,TS Mills,0,0,0,None,None
2,1082591,IPL,2017-04-05,Hyderabad,Sunrisers Hyderabad,0,None,DA Warner,S Dhawan,TS Mills,4,0,4,None,None
3,1082591,IPL,2017-04-05,Hyderabad,Sunrisers Hyderabad,0,None,DA Warner,S Dhawan,TS Mills,0,0,0,None,None
4,1082591,IPL,2017-04-05,Hyderabad,Sunrisers Hyderabad,0,None,DA Warner,S Dhawan,TS Mills,0,2,2,None,None



Matches missing values:
match_id                0
match_type              0
date                    0
city                  634
venue                   0
team1                   0
team2                   0
toss_winner             0
toss_decision           0
winner                476
result_type          9142
result_by_runs       4975
result_by_wickets    5122
dtype: int64

Deliveries missing values (top):
ball_id        4601993
player_out     4601993
wicket_kind    4601993
city            422071
match_type           0
date                 0
match_id             0
over                 0
team                 0
batter               0
non_striker          0
runs_batter          0
bowler               0
runs_total           0
runs_extras          0
dtype: int64


In [13]:
print(matches_df['winner'].isnull().sum(), "matches without winner info")
print(matches_df['city'].isnull().sum(), "matches without city info")
print(deliveries_df[['batter','bowler']].isnull().sum())


476 matches without winner info
634 matches without city info
batter    0
bowler    0
dtype: int64


In [15]:
# Step 1: Handle missing winners
matches_df['winner'] = matches_df['winner'].fillna('No result')

# Step 2: Map venue → city (only where city is missing)
venue_city_map = matches_df.dropna(subset=['venue', 'city']).set_index('venue')['city'].to_dict()

matches_df['city'] = matches_df.apply(
    lambda x: venue_city_map.get(x['venue'], x['city']), axis=1
)

# Step 3: Create match result column
def get_result(row):
    if pd.notna(row.get('result_by_runs')):
        return f"{int(row['result_by_runs'])} runs"
    elif pd.notna(row.get('result_by_wickets')):
        return f"{int(row['result_by_wickets'])} wickets"
    elif pd.notna(row.get('result_type')):
        return row['result_type']
    else:
        return "Unknown"

matches_df['match_result'] = matches_df.apply(get_result, axis=1)

# Step 4: Add ball_id to deliveries (serial ball number in each match)
deliveries_df['ball_id'] = deliveries_df.groupby('match_id').cumcount() + 1


In [16]:
# Re-check for missing cities after mapping
missing_city_mask = matches_df['city'].isna() | (matches_df['city'] == "")

# Fill remaining missing cities with "Unknown"
matches_df.loc[missing_city_mask, 'city'] = "Unknown"


In [17]:
matches_df['city'].value_counts().head(10)   # top 10 most common cities
matches_df['city'].eq("Unknown").sum()      # count of Unknown cities


np.int64(41)

In [22]:
import pandas as pd
from sqlalchemy import create_engine, String, Integer, Date
from datetime import datetime

# Ensure your date columns are in datetime.date format
matches_df['date'] = pd.to_datetime(matches_df['date']).dt.date
deliveries_df['date'] = pd.to_datetime(deliveries_df['date']).dt.date

# Create SQLite engine
engine = create_engine('sqlite:///cricsheet.db', echo=True)

# Save matches_df to SQLite
matches_df.to_sql(
    'matches',
    con=engine,
    if_exists='replace',
    index=False,
    dtype={
        'match_id': String,
        'match_type': String,
        'date': Date,
        'city': String,
        'venue': String,
        'team1': String,
        'team2': String,
        'toss_winner': String,
        'toss_decision': String,
        'winner': String,
        'result_type': String,
        'result_by_runs': Integer,
        'result_by_wickets': Integer,
        'match_result': String
    }
)

# Save deliveries_df to SQLite
deliveries_df.to_sql(
    'deliveries',
    con=engine,
    if_exists='replace',
    index=False,
    dtype={
        'match_id': String,
        'ball_id': Integer,
        'date': Date,
        'city': String,
        'team': String,
        'over': Integer,
        'batter': String,
        'non_striker': String,
        'bowler': String,
        'runs_batter': Integer,
        'runs_extras': Integer,
        'runs_total': Integer,
        'wicket_kind': String,
        'player_out': String
    }
)

print("DataFrames saved to SQLite database successfully!")


2025-09-13 10:03:11,930 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-13 10:03:11,939 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("matches")
2025-09-13 10:03:11,941 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:03:11,941 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("matches")
2025-09-13 10:03:11,941 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:03:11,941 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-09-13 10:03:11,941 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:03:11,950 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-09-13 10:03:11,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:03:11,954 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("matches")
2025-09-13 10:03:11,957 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 1

In [23]:
import pandas as pd

# Read matches
matches_sql = pd.read_sql("SELECT * FROM matches LIMIT 5", con=engine)
print(matches_sql)

# Read deliveries
deliveries_sql = pd.read_sql("SELECT * FROM deliveries LIMIT 5", con=engine)
print(deliveries_sql)


2025-09-13 10:06:10,040 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-13 10:06:10,040 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM matches LIMIT 5")
2025-09-13 10:06:10,043 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:06:10,044 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM matches LIMIT 5")
2025-09-13 10:06:10,044 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:06:10,044 INFO sqlalchemy.engine.Engine SELECT * FROM matches LIMIT 5
2025-09-13 10:06:10,051 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:06:10,101 INFO sqlalchemy.engine.Engine ROLLBACK
  match_id match_type        date       city  \
0  1082591        IPL  2017-04-05  Hyderabad   
1  1082592        IPL  2017-04-06       Pune   
2  1082593        IPL  2017-04-07     Rajkot   
3  1082594        IPL  2017-04-08     Indore   
4  1082595        IPL  2017-04-08  Bengaluru   

                                       venue                        team

In [25]:
from sqlalchemy import Index, Table, MetaData

metadata = MetaData()  # no bind here
matches_table = Table('matches', metadata, autoload_with=engine)
deliveries_table = Table('deliveries', metadata, autoload_with=engine)

# Create indexes for faster querying
Index('idx_matches_match_id', matches_table.c.match_id).create(engine)
Index('idx_matches_winner', matches_table.c.winner).create(engine)
Index('idx_deliveries_match_id', deliveries_table.c.match_id).create(engine)
Index('idx_deliveries_ball_id', deliveries_table.c.ball_id).create(engine)
Index('idx_deliveries_team', deliveries_table.c.team).create(engine)


2025-09-13 10:09:21,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-09-13 10:09:21,566 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("matches")
2025-09-13 10:09:21,566 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:09:21,570 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2025-09-13 10:09:21,570 INFO sqlalchemy.engine.Engine [raw sql] ('matches',)
2025-09-13 10:09:21,577 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("matches")
2025-09-13 10:09:21,577 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:09:21,579 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("matches")
2025-09-13 10:09:21,581 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-09-13 10:09:21,581 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 

In [26]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to SQLite database
engine = create_engine('sqlite:///cricsheet.db')

# Quick test: show first 5 rows of matches
matches_df = pd.read_sql("SELECT * FROM matches LIMIT 5", con=engine)
matches_df


,match_id,match_type,date,city,venue,team1,team2,toss_winner,toss_decision,winner,result_type,result_by_runs,result_by_wickets,match_result
0,1082591,IPL,2017-04-05,Hyderabad,"Rajiv Gandhi International Stadium, Uppal",Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,Sunrisers Hyderabad,None,35.0,NaN,35 runs
1,1082592,IPL,2017-04-06,Pune,Maharashtra Cricket Association Stadium,Rising Pune Supergiant,Mumbai Indians,Rising Pune Supergiant,field,Rising Pune Supergiant,None,NaN,7.0,7 wickets
2,1082593,IPL,2017-04-07,Rajkot,Saurashtra Cricket Association Stadium,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,Kolkata Knight Riders,None,NaN,10.0,10 wickets
3,1082594,IPL,2017-04-08,Indore,Holkar Cricket Stadium,Kings XI Punjab,Rising Pune Supergiant,Kings XI Punjab,field,Kings XI Punjab,None,NaN,6.0,6 wickets
4,1082595,IPL,2017-04-08,Bengaluru,M.Chinnaswamy Stadium,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,Royal Challengers Bangalore,None,15.0,NaN,15 runs


In [56]:
import os
import pandas as pd
import json
from sqlalchemy import create_engine, String, Integer, Date

# ---------- 1️⃣ Set paths ----------
data_folder = r"C:\py project\cricsheet_project\data\Raw"  # change if needed
db_path = 'sqlite:///cricsheet.db'  # SQLite database file

# ---------- 2️⃣ Parse JSONs into DataFrames ----------
def normalize_string(s):
    return str(s).strip() if s else None

def parse_folder(folder_path, match_type):
    matches = []
    deliveries = []

    files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    for fname in files:
        fp = os.path.join(folder_path, fname)
        with open(fp, 'r', encoding='utf-8') as f:
            data = json.load(f)

        info = data.get('info', {})
        match_id = os.path.splitext(fname)[0]
        teams = info.get('teams', [])

        dates = info.get('dates')
        date = pd.to_datetime(dates[0]).date() if isinstance(dates, list) else None

        outcome = info.get('outcome', {})
        by = outcome.get('by', {})
        result_by_runs = by.get('runs') if isinstance(by, dict) else None
        result_by_wickets = by.get('wickets') if isinstance(by, dict) else None

        matches.append({
            'match_id': match_id,
            'match_type': match_type,
            'date': date,
            'team1': teams[0] if len(teams)>0 else None,
            'team2': teams[1] if len(teams)>1 else None,
            'winner': outcome.get('winner'),
            'result_type': outcome.get('result'),
            'result_by_runs': result_by_runs,
            'result_by_wickets': result_by_wickets
        })

        innings_list = data.get('innings', [])
        for inning in innings_list:
            if 'overs' not in inning:
                inner = next(iter(inning.values()), None)
                inning_dict = inner if isinstance(inner, dict) else {}
            else:
                inning_dict = inning

            team = inning_dict.get('team')
            for over in inning_dict.get('overs', []):
                over_number = over.get('over')
                for delivery in over.get('deliveries', []):
                    # handle nested dict
                    d = next(iter(delivery.values())) if not 'batter' in delivery else delivery
                    runs = d.get('runs', {})
                    wicket = d.get('wicket', {})
                    deliveries.append({
                        'match_id': match_id,
                        'match_type': match_type,
                        'date': date,
                        'team': team,
                        'over': over_number,
                        'batter': normalize_string(d.get('batter')),
                        'bowler': normalize_string(d.get('bowler')),
                        'runs_batter': runs.get('batter', 0),
                        'runs_extras': runs.get('extras', 0),
                        'runs_total': runs.get('total', 0),
                        'wicket_kind': wicket.get('kind') if isinstance(wicket, dict) else None,
                        'player_out': normalize_string(wicket.get('player_out')) if isinstance(wicket, dict) else None
                    })

    return pd.DataFrame(matches), pd.DataFrame(deliveries)

# ---------- 3️⃣ Loop over match types ----------
all_matches = []
all_deliveries = []

folders = ['IPL','ODI','T20','TEST']

for mtype in folders:
    folder = os.path.join(data_folder, mtype)
    if os.path.isdir(folder):
        print(f"Parsing {mtype} folder: {folder}")
        m_df, d_df = parse_folder(folder, mtype)
        all_matches.append(m_df)
        all_deliveries.append(d_df)

matches_df = pd.concat(all_matches, ignore_index=True)
deliveries_df = pd.concat(all_deliveries, ignore_index=True)

print("Matches:", matches_df.shape)
print("Deliveries:", deliveries_df.shape)

# ---------- 4️⃣ Save to SQLite ----------
engine = create_engine(db_path)

matches_df.to_sql('matches', con=engine, if_exists='replace', index=False,
                  dtype={
                      'match_id': String,
                      'match_type': String,
                      'date': Date,
                      'team1': String,
                      'team2': String,
                      'winner': String,
                      'result_type': String,
                      'result_by_runs': Integer,
                      'result_by_wickets': Integer
                  })

deliveries_df.to_sql('deliveries', con=engine, if_exists='replace', index=False,
                     dtype={
                         'match_id': String,
                         'match_type': String,
                         'date': Date,
                         'team': String,
                         'over': Integer,
                         'batter': String,
                         'bowler': String,
                         'runs_batter': Integer,
                         'runs_extras': Integer,
                         'runs_total': Integer,
                         'wicket_kind': String,
                         'player_out': String
                     })

print("DataFrames saved to SQLite successfully!")

# ---------- 5️⃣ Quick test queries ----------
print(pd.read_sql("SELECT * FROM deliveries LIMIT 5", con=engine))
print(pd.read_sql("SELECT * FROM matches LIMIT 5", con=engine))


Parsing IPL folder: C:\py project\cricsheet_project\data\Raw\IPL
Parsing ODI folder: C:\py project\cricsheet_project\data\Raw\ODI
Parsing T20 folder: C:\py project\cricsheet_project\data\Raw\T20
Parsing TEST folder: C:\py project\cricsheet_project\data\Raw\TEST
Matches: (9618, 9)
Deliveries: (4601993, 12)
DataFrames saved to SQLite successfully!
  match_id match_type        date                 team  over     batter  \
0  1082591        IPL  2017-04-05  Sunrisers Hyderabad     0  DA Warner   
1  1082591        IPL  2017-04-05  Sunrisers Hyderabad     0  DA Warner   
2  1082591        IPL  2017-04-05  Sunrisers Hyderabad     0  DA Warner   
3  1082591        IPL  2017-04-05  Sunrisers Hyderabad     0  DA Warner   
4  1082591        IPL  2017-04-05  Sunrisers Hyderabad     0  DA Warner   

     bowler  runs_batter  runs_extras  runs_total wicket_kind player_out  
0  TS Mills            0            0           0        None       None  
1  TS Mills            0            0           0  

In [90]:
import os
import json
import pandas as pd
import logging
from sqlalchemy import create_engine, String, Integer, Date
from datetime import datetime
import traceback

logging.basicConfig(filename='cricsheet_parse.log', level=logging.INFO,
                    format='%(asctime)s %(levelname)s: %(message)s')

def normalize_string(s):
    return str(s).strip() if s else None

def parse_single_match(file_path, match_type):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    info = data.get('info', {}) or {}
    match_id = os.path.splitext(os.path.basename(file_path))[0]

    # handle date
    dates = info.get('dates')
    if isinstance(dates, list) and dates: date = dates[0]
    elif isinstance(dates, str): date = dates
    else: date = None

    city = info.get('city')
    venue = info.get('venue')
    teams = info.get('teams') or []
    toss = info.get('toss') or {}
    outcome = info.get('outcome') or {}
    winner = outcome.get('winner')
    result_type = outcome.get('result')
    by = outcome.get('by')
    result_by_runs = by.get('runs') if isinstance(by, dict) else None
    result_by_wickets = by.get('wickets') if isinstance(by, dict) else None

    match_row = {
        'match_id': match_id,
        'match_type': match_type,
        'date': date,
        'city': city,
        'venue': venue,
        'team1': teams[0] if len(teams) > 0 else None,
        'team2': teams[1] if len(teams) > 1 else None,
        'toss_winner': toss.get('winner'),
        'toss_decision': toss.get('decision'),
        'winner': winner,
        'result_type': result_type,
        'result_by_runs': result_by_runs,
        'result_by_wickets': result_by_wickets
    }

    deliveries = []
    innings_list = data.get('innings', []) or []

    for inning in innings_list:
        inning_dict = inning.get('overs') is None and next(iter(inning.values()), {}) or inning
        team = inning_dict.get('team')
        for over in inning_dict.get('overs', []):
            over_number = over.get('over')
            for delivery in over.get('deliveries', []):
                if 'batter' not in delivery:
                    delivery = next(iter(delivery.values()))
                runs = delivery.get('runs', {})
                wicket_obj = delivery.get('wicket') if isinstance(delivery.get('wicket'), dict) else {}
                deliveries.append({
                    'match_id': match_id,
                    'match_type': match_type,
                    'date': date,
                    'city': city,
                    'team': team,
                    'over': over_number,
                    'ball_id': None,
                    'batter': normalize_string(delivery.get('batter')),
                    'non_striker': normalize_string(delivery.get('non_striker')),
                    'bowler': normalize_string(delivery.get('bowler')),
                    'runs_batter': runs.get('batter', 0),
                    'runs_extras': runs.get('extras', 0),
                    'runs_total': runs.get('total', runs.get('batter',0)+runs.get('extras',0)),
                    'wicket_kind': wicket_obj.get('kind'),
                    'player_out': normalize_string(wicket_obj.get('player_out'))
                })

    return match_row, deliveries

def parse_folder(folder_path, match_type):
    matches, deliveries = [], []
    files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    for f in files:
        try:
            m, d = parse_single_match(os.path.join(folder_path,f), match_type)
            matches.append(m)
            deliveries.extend(d)
        except Exception as e:
            logging.error(f"Failed parsing {f}: {e}\n{traceback.format_exc()}")
    return pd.DataFrame(matches), pd.DataFrame(deliveries)

# Paths to your folders
folders = {
    'IPL': r"C:\py project\cricsheet_project\data\Raw\IPL",
    'ODI': r"C:\py project\cricsheet_project\data\Raw\ODI",
    'T20': r"C:\py project\cricsheet_project\data\Raw\T20",
    'TEST': r"C:\py project\cricsheet_project\data\Raw\TEST"
}

all_matches, all_deliveries = [], []

for mtype, folder in folders.items():
    if not os.path.isdir(folder):
        print(f"Folder not found: {folder}, skipping {mtype}")
        continue
    print(f"Parsing {folder} as {mtype}")
    m_df, d_df = parse_folder(folder, mtype)
    all_matches.append(m_df)
    all_deliveries.append(d_df)
    print(f"  -> matches: {len(m_df)}, deliveries: {len(d_df)}")

matches_df = pd.concat(all_matches, ignore_index=True) if all_matches else pd.DataFrame()
deliveries_df = pd.concat(all_deliveries, ignore_index=True) if all_deliveries else pd.DataFrame()

# Fill missing winners and city safely
if 'winner' in matches_df.columns:
    matches_df['winner'] = matches_df['winner'].fillna('No Result')
if 'city' in matches_df.columns:
    matches_df['city'] = matches_df['city'].fillna('Unknown')
else:
    matches_df['city'] = 'Unknown'
if 'venue' not in matches_df.columns:
    matches_df['venue'] = None

# Convert dates
matches_df['date'] = pd.to_datetime(matches_df['date'], errors='coerce').dt.date
deliveries_df['date'] = pd.to_datetime(deliveries_df['date'], errors='coerce').dt.date

# SQLite save
engine = create_engine('sqlite:///cricsheet.db', echo=False)
matches_df.to_sql('matches', engine, if_exists='replace', index=False)
deliveries_df.to_sql('deliveries', engine, if_exists='replace', index=False)

print("Matches and deliveries saved safely to SQLite!")

# Quick column check
print("Matches columns:", matches_df.columns.tolist())
print("Deliveries columns:", deliveries_df.columns.tolist())
print(matches_df.head())
print(deliveries_df.head())


Parsing C:\py project\cricsheet_project\data\Raw\IPL as IPL
  -> matches: 1169, deliveries: 278205
Parsing C:\py project\cricsheet_project\data\Raw\ODI as ODI
  -> matches: 3000, deliveries: 1588920
Parsing C:\py project\cricsheet_project\data\Raw\T20 as T20
  -> matches: 4570, deliveries: 1034413
Parsing C:\py project\cricsheet_project\data\Raw\TEST as TEST
  -> matches: 879, deliveries: 1700455
Matches and deliveries saved safely to SQLite!
Matches columns: ['match_id', 'match_type', 'date', 'city', 'venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner', 'result_type', 'result_by_runs', 'result_by_wickets']
Deliveries columns: ['match_id', 'match_type', 'date', 'city', 'team', 'over', 'ball_id', 'batter', 'non_striker', 'bowler', 'runs_batter', 'runs_extras', 'runs_total', 'wicket_kind', 'player_out']
  match_id match_type        date       city  \
0  1082591        IPL  2017-04-05  Hyderabad   
1  1082592        IPL  2017-04-06       Pune   
2  1082593        IPL  2017-

In [105]:
# 1️⃣ Top 10 Batsmen by total runs
query1 = """
-- Top 10 Batsmen by total runs
SELECT batter, match_type, SUM(runs_batter) AS total_runs
FROM deliveries
GROUP BY batter, match_type
ORDER BY total_runs DESC
LIMIT 10;
"""



# 3️⃣ Team wins by match type
query3 = """
-- Team wins by match type
SELECT COALESCE(winner, 'No Result') AS team, match_type, COUNT(*) AS wins
FROM matches
GROUP BY team, match_type
ORDER BY wins DESC;
"""

# 4️⃣ Average margin of victory
query4 = """
-- Average margin of victory
SELECT match_type, 
       AVG(COALESCE(runs_margin,0)) AS avg_runs_margin,
       AVG(COALESCE(wickets_margin,0)) AS avg_wickets_margin
FROM matches
GROUP BY match_type;
"""

# 5️⃣ Matches per year
query5 = """
-- Matches per year
SELECT strftime('%Y', date) AS year, match_type, COUNT(*) AS matches_count
FROM matches
GROUP BY year, match_type
ORDER BY year;
"""

# 6️⃣ Most common venues
query6 = """
-- Most common venues
SELECT COALESCE(venue,'Unknown') AS venue, match_type, COUNT(*) AS matches_count
FROM matches
GROUP BY venue, match_type
ORDER BY matches_count DESC
LIMIT 10;
"""

# 7️⃣ Most frequent toss winners
query7 = """
-- Most frequent toss winners
SELECT COALESCE(toss_winner,'Unknown') AS toss_winner, match_type, COUNT(*) AS count
FROM matches
GROUP BY toss_winner, match_type
ORDER BY count DESC
LIMIT 10;
"""

# 8️⃣ Most common toss decisions
query8 = """
-- Most common toss decisions
SELECT COALESCE(toss_decision,'Unknown') AS toss_decision, match_type, COUNT(*) AS count
FROM matches
GROUP BY toss_decision, match_type
ORDER BY count DESC;
"""

# 9️⃣ Matches per city
query9 = """
-- Matches per city (handles missing city)
SELECT COALESCE(city,'Unknown') AS city, match_type, COUNT(*) AS matches_count
FROM matches
GROUP BY city, match_type
ORDER BY matches_count DESC;
"""




# 12️⃣ Total extras per match type
query12 = """
-- Total extras per match type
SELECT match_type, SUM(COALESCE(runs_extras,0)) AS total_extras
FROM deliveries
GROUP BY match_type
ORDER BY total_extras DESC;
"""

# 13️⃣ Matches won by batting first vs second
query13 = """
-- Matches won by batting first vs second
SELECT 
  CASE 
    WHEN toss_decision='bat' AND winner=team1 THEN 'Bat First Win'
    WHEN toss_decision='field' AND winner=team2 THEN 'Bat First Win'
    ELSE 'Other'
  END AS result_type,
  COUNT(*) AS matches_count
FROM matches
GROUP BY result_type;
"""



# 15️⃣ Top scoring teams by match type
query15 = """
-- Top scoring teams by match type
SELECT team, match_type, SUM(COALESCE(runs_total,0)) AS total_runs
FROM deliveries
GROUP BY team, match_type
ORDER BY total_runs DESC
LIMIT 10;
"""

# 16️⃣ Total balls bowled by each bowler
query16 = """
-- Total balls bowled by each bowler
SELECT bowler, match_type, COUNT(*) AS balls_bowled
FROM deliveries
WHERE bowler IS NOT NULL
GROUP BY bowler, match_type
ORDER BY balls_bowled DESC
LIMIT 10;
"""

# 17️⃣ Most frequent non-strikers
query17 = """
-- Most frequent non-strikers
SELECT non_striker, match_type, COUNT(*) AS count
FROM deliveries
WHERE non_striker IS NOT NULL
GROUP BY non_striker, match_type
ORDER BY count DESC
LIMIT 10;
"""

# 18️⃣ Average runs per ball by match type
query18 = """
-- Average runs per ball by match type
SELECT match_type, AVG(COALESCE(runs_total,0)) AS avg_runs_per_ball
FROM deliveries
GROUP BY match_type;
"""

# 19️⃣ Matches played by each team
query19 = """
-- Matches played by each team
SELECT team, match_type, COUNT(DISTINCT match_id) AS matches_played
FROM deliveries
GROUP BY team, match_type
ORDER BY matches_played DESC
LIMIT 10;
"""




In [106]:
for title, sql in queries.items():
    print(f"\n===== {title} =====")
    try:
        df = pd.read_sql(sql, con=engine)
        print(df)
    except Exception as e:
        print(f"Error executing query: {e}")


===== top_batsmen =====
           batter match_type  total_runs
0         V Kohli        ODI       14059
1         JE Root       TEST       13543
2         AN Cook       TEST       12472
3   KC Sangakkara        ODI       11618
4       RG Sharma        ODI       10948
5       SPD Smith       TEST       10477
6        MS Dhoni        ODI       10274
7  AB de Villiers        ODI        9435
8   KS Williamson       TEST        9276
9         V Kohli       TEST        9230

===== top_bowlers =====
Empty DataFrame
Columns: [bowler, match_type, wickets]
Index: []

===== team_wins =====
                        team match_type  wins
0                  Australia        ODI   383
1                      India        ODI   374
2                    England        ODI   302
3               South Africa        ODI   298
4                  Sri Lanka        ODI   245
..                       ...        ...   ...
156                   Gambia        T20     1
157             ICC World XI        T20    

In [109]:
import sqlite3
import pandas as pd

# Connect to (or create) the SQLite database
conn = sqlite3.connect(r"C:\py project\cricsheet_project\cricsheet.sqlite")

# Save matches and deliveries to SQLite
matches_df.to_sql("matches", conn, if_exists="replace", index=False)
deliveries_df.to_sql("deliveries", conn, if_exists="replace", index=False)

print("Tables saved to SQLite successfully!")


Tables saved to SQLite successfully!


In [110]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in DB:", tables)


Tables in DB:          name
0     matches
1  deliveries


In [113]:
import pandas as pd
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect("cricsheet.db")  # replace with your DB path

# Top 10 bowlers by balls bowled
top_bowlers = pd.read_sql("""
    SELECT bowler, match_type, COUNT(*) AS balls_bowled
    FROM deliveries
    WHERE bowler IS NOT NULL
    GROUP BY bowler, match_type
    ORDER BY balls_bowled DESC
    LIMIT 10;
""", conn)

print(top_bowlers)


         bowler match_type  balls_bowled
0   JM Anderson       TEST         39114
1       NM Lyon       TEST         34507
2     SCJ Broad       TEST         33896
3      R Ashwin       TEST         27140
4  HMRKB Herath       TEST         23563
5    TG Southee       TEST         23557
6      I Sharma       TEST         19461
7     RA Jadeja       TEST         19337
8      MA Starc       TEST         19231
9      DW Steyn       TEST         18172


In [120]:
import pandas as pd
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect("cricsheet.db")

# 1️⃣ Most frequent non-strikers
most_frequent_non_strikers = pd.read_sql("""
    SELECT non_striker, match_type, COUNT(*) AS appearances
    FROM deliveries
    WHERE non_striker IS NOT NULL
    GROUP BY non_striker, match_type
    ORDER BY appearances DESC
    LIMIT 10;
""", conn)

# 2️⃣ Highest individual scores (per innings)
highest_individual_scores = pd.read_sql("""
    SELECT batter, match_type, MAX(runs_batter) AS highest_score
    FROM deliveries
    GROUP BY batter, match_type
    ORDER BY highest_score DESC
    LIMIT 10;
""", conn)

# 3️⃣ Matches with most total runs
matches_most_runs = pd.read_sql("""
    SELECT match_id, match_type, SUM(runs_total) AS total_match_runs
    FROM deliveries
    GROUP BY match_id, match_type
    ORDER BY total_match_runs DESC
    LIMIT 10;
""", conn)

# 4️⃣ Average wickets per match per format
avg_wickets_per_match = pd.read_sql("""
    SELECT match_type, 
           ROUND(SUM(CASE WHEN player_out IS NOT NULL THEN 1 ELSE 0 END)*1.0/COUNT(DISTINCT match_id), 2) AS avg_wickets
    FROM deliveries
    GROUP BY match_type
    ORDER BY avg_wickets DESC;
""", conn)

# Display results
print("===== Most Frequent Non-Strikers =====")
print(most_frequent_non_strikers, "\n")

print("===== Highest Individual Scores =====")
print(highest_individual_scores, "\n")

print("===== Matches with Most Total Runs =====")
print(matches_most_runs, "\n")

print("===== Average Wickets per Match =====")
print(avg_wickets_per_match, "\n")

# Close connection
conn.close()


===== Most Frequent Non-Strikers =====
     non_striker match_type  appearances
0        AN Cook       TEST        27143
1        JE Root       TEST        25217
2      SPD Smith       TEST        20536
3        HM Amla       TEST        18805
4  KS Williamson       TEST        18214
5      Azhar Ali       TEST        17198
6     AD Mathews       TEST        16548
7        V Kohli       TEST        16387
8  KC Sangakkara       TEST        16142
9      CA Pujara       TEST        16064 

===== Highest Individual Scores =====
              batter match_type  highest_score
0          A Symonds       TEST              8
1          BV Vitori        T20              7
2         CJL Rogers       TEST              7
3          GH Worker        T20              7
4      KC Brathwaite       TEST              7
5          MJ Clarke       TEST              7
6            T Taibu        ODI              7
7           WA Young       TEST              7
8  A Aitken-Drummond        T20              6


In [124]:
import pandas as pd
import sqlite3
import json

conn = sqlite3.connect("cricsheet.db")

# Load deliveries table
df = pd.read_sql("SELECT * FROM deliveries", conn)

# Extract batter and total runs from JSON
if 'runs' in df.columns:
    df['runs_batter'] = df['runs'].apply(lambda x: x['batter'] if isinstance(x, dict) else 0)
    df['runs_total']  = df['runs'].apply(lambda x: x['total'] if isinstance(x, dict) else 0)

# Save back to DB if you want SQL queries to work
df.to_sql("deliveries_clean", conn, if_exists="replace", index=False)


4601993

In [126]:
import sqlite3
conn = sqlite3.connect("cricsheet.db")
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(deliveries);")
columns = cursor.fetchall()
for col in columns:
    print(col)


(0, 'match_id', 'TEXT', 0, None, 0)
(1, 'match_type', 'TEXT', 0, None, 0)
(2, 'date', 'DATE', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'team', 'TEXT', 0, None, 0)
(5, 'over', 'BIGINT', 0, None, 0)
(6, 'ball_id', 'TEXT', 0, None, 0)
(7, 'batter', 'TEXT', 0, None, 0)
(8, 'non_striker', 'TEXT', 0, None, 0)
(9, 'bowler', 'TEXT', 0, None, 0)
(10, 'runs_batter', 'BIGINT', 0, None, 0)
(11, 'runs_extras', 'BIGINT', 0, None, 0)
(12, 'runs_total', 'BIGINT', 0, None, 0)
(13, 'wicket_kind', 'TEXT', 0, None, 0)
(14, 'player_out', 'TEXT', 0, None, 0)


In [140]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("cricsheet.db")

# List of 20 queries with descriptive names
queries = {
    "01. Top 10 Batsmen by Total Runs": """
        SELECT batter, SUM(runs_batter) AS total_runs
        FROM deliveries
        GROUP BY batter
        ORDER BY total_runs DESC
        LIMIT 10;
    """,

    "02. Highest Team Scores per Match": """
        SELECT match_id, team, SUM(runs_total) AS total_runs
        FROM deliveries
        GROUP BY match_id, team
        ORDER BY total_runs DESC
        LIMIT 10;
    """,

    "03. Matches Won by Each Team": """
        SELECT winner, COUNT(*) AS wins
        FROM matches
        WHERE winner IS NOT NULL
        GROUP BY winner
        ORDER BY wins DESC;
    """,

    "04. Matches Per Year": """
        SELECT strftime('%Y', date) AS year, COUNT(*) AS matches_count
        FROM matches
        GROUP BY year
        ORDER BY year;
    """,

    "05. Toss Wins vs Match Wins": """
        SELECT toss_winner, winner, COUNT(*) AS count
        FROM matches
        GROUP BY toss_winner, winner
        ORDER BY count DESC;
    """,

    "06. Most Common Toss Decision": """
        SELECT toss_decision, COUNT(*) AS frequency
        FROM matches
        GROUP BY toss_decision
        ORDER BY frequency DESC;
    """,

    "07. Average Runs Per Over by Team": """
        SELECT team,
               ROUND(SUM(runs_total)*1.0/COUNT(DISTINCT match_id || '-' || over), 3) AS avg_runs_per_over
        FROM deliveries
        GROUP BY team
        ORDER BY avg_runs_per_over DESC;
    """,

    "08. Top Batsmen by Average Runs": """
        SELECT batter, ROUND(SUM(runs_batter)*1.0/COUNT(DISTINCT match_id), 2) AS batting_avg
        FROM deliveries
        GROUP BY batter
        ORDER BY batting_avg DESC
        LIMIT 10;
    """,

    "09. Top Teams by Total Runs": """
        SELECT team, SUM(runs_total) AS total_runs
        FROM deliveries
        GROUP BY team
        ORDER BY total_runs DESC
        LIMIT 10;
    """,

    "10. Top Batsmen by Boundaries": """
        SELECT batter,
               SUM(CASE WHEN runs_batter=4 THEN 1 ELSE 0 END) AS fours,
               SUM(CASE WHEN runs_batter=6 THEN 1 ELSE 0 END) AS sixes
        FROM deliveries
        GROUP BY batter
        ORDER BY (fours + sixes) DESC
        LIMIT 10;
    """,

    "11. Matches per City": """
        SELECT city, COUNT(*) AS matches_count
        FROM matches
        GROUP BY city
        ORDER BY matches_count DESC
        LIMIT 10;
    """,

    "12. Average Runs per Over by Venue": """
        SELECT m.venue,
       ROUND(SUM(d.runs_total)*1.0/COUNT(DISTINCT d.match_id || '-' || d.over), 2) AS avg_runs_per_over
FROM deliveries d
JOIN matches m ON d.match_id = m.match_id
WHERE m.venue IS NOT NULL
GROUP BY m.venue
ORDER BY avg_runs_per_over DESC
LIMIT 10;
""",

    "13. Top Bowlers by Economy Rate": """
        SELECT bowler,
               ROUND(SUM(runs_total)*6.0/COUNT(*), 2) AS economy_rate
        FROM deliveries
        WHERE bowler IS NOT NULL
        GROUP BY bowler
        ORDER BY economy_rate ASC
        LIMIT 10;
    """,

    "14. Total Sixes by Batter": """
        SELECT batter, COUNT(*) AS sixes
        FROM deliveries
        WHERE runs_batter = 6
        GROUP BY batter
        ORDER BY sixes DESC
        LIMIT 10;
    """,

    "15. Total Fours by Batter": """
        SELECT batter, COUNT(*) AS fours
        FROM deliveries
        WHERE runs_batter = 4
        GROUP BY batter
        ORDER BY fours DESC
        LIMIT 10;
    """,

    "16. Most Successful Teams by Match Type": """
        SELECT match_type, winner, COUNT(*) AS wins
        FROM matches
        WHERE winner IS NOT NULL
        GROUP BY match_type, winner
        ORDER BY match_type, wins DESC;
    """,

    "17. Matches with No Result": """
        SELECT COUNT(*) AS no_result_matches
        FROM matches
        WHERE winner IS NULL;
    """,

    "18. Team Win % when Batting First": """
        SELECT winner,
               ROUND(100.0*SUM(CASE WHEN toss_winner = winner THEN 1 ELSE 0 END)/COUNT(*),2) AS win_percent
        FROM matches
        GROUP BY winner
        ORDER BY win_percent DESC;
    """,

    "19. Top 10 Teams by Average Runs per Match": """
        SELECT team, ROUND(SUM(runs_total)*1.0/COUNT(DISTINCT match_id), 2) AS avg_runs_per_match
        FROM deliveries
        GROUP BY team
        ORDER BY avg_runs_per_match DESC
        LIMIT 10;
    """,

    "20. Batsmen with Most Dot Balls": """
        SELECT batter, COUNT(*) AS dot_balls
        FROM deliveries
        WHERE runs_batter = 0
        GROUP BY batter
        ORDER BY dot_balls DESC
        LIMIT 10;
    """
}

# Execute all queries and print results
for title, query in queries.items():
    print(f"\n===== {title} =====")
    try:
        df = pd.read_sql(query, conn)
        print(df)
    except Exception as e:
        print(f"Error executing query: {e}")

# Close the connection
conn.close()



===== 01. Top 10 Batsmen by Total Runs =====
           batter  total_runs
0         V Kohli       35929
1       RG Sharma       26339
2       DA Warner       25230
3  AB de Villiers       24389
4   KC Sangakkara       23176
5        MS Dhoni       22018
6         JE Root       21326
7   KS Williamson       21069
8       SPD Smith       19730
9         HM Amla       18829

===== 02. Highest Team Scores per Match =====
  match_id        team  total_runs
0   233797    Pakistan        1078
1  1448350       India        1014
2   892511   Australia         944
3  1330871     England         921
4    64062       India         916
5   297808       India         910
6   602472   Sri Lanka         905
7   690349   Sri Lanka         892
8   858493  Bangladesh         887
9   352663     England         879

===== 03. Matches Won by Each Team =====
           winner  wins
0       Australia   740
1           India   719
2         England   649
3    South Africa   563
4     New Zealand   490
..    

In [ ]:
# Quick sanity checks
print(matches.info())
print(deliveries.info())

# Check missing values
print(matches.isna().sum()
print(deliveries.isna().sum())


NameError: name 'matches' is not defined